In [2]:
import os, sys, time
import matplotlib.pyplot as plt


import numpy as np
import scipy as sp
import itertools as itr
from pprint import pprint
from scipy import fftpack, ndimage
from scipy.misc import imread
from scipy.signal.signaltools import correlate2d as cor2d


def do_grayscale(imgarr):
    if len(imgarr.shape) == 3:
        return sp.average(imgarr, -1)
    else:
        return imgarr

def do_imgtrueflat(ipath):
    idata = ndimage.imread(ipath, flatten=True)
    return idata

def do_imgzncc(ipath):
    imgdata = do_imgtrueflat(ipath)
    zndata = (imgdata - imgdata.mean())/ imgdata.std()
    return zndata
    
def do_imgfft2(ipath):
    imgdata = do_imgtrueflat(ipath)
    fft2dat = fftpack.fft2(imgdata)
    return fft2dat

def do_znccfft2(ipath):
    imgdata = do_imgtrueflat(ipath)
    zndata = (imgdata - imgdata.mean())/ imgdata.std()
    fft2dat = fftpack.fft2(zndata)
    return fft2dat
    
def doplt(ifftdata):
#     plt.imshow(20*np.log10(abs(ifftdata)))
    from matplotlib.colors import LogNorm
    plt.figure(figsize=(8, 6), dpi=80)
    plt.imshow(np.abs(ifftdata), norm=LogNorm(vmin=2))
#     plt.colorbar()
    plt.show()

#>******************************************************************************
def fft2_crosscorr(imgx_dat, imgy_dat):
    """ Assume imgx_dat is always going to be the bigger image, area wise"""
    imgx_dim = imgx_dat.shape
    imgy_dim = imgy_dat.shape
    ixd, iyd = imgx_dim, imgy_dim
    xht = ixd[0] # heigth 
    xwd = ixd[1] # width
    yht = iyd[0] # h
    ywd = iyd[1] # w

    divnum = 0
    mulfact = 4
    upperb, lowerb = 0, 0
    
    locmax_lt = []
    locnmax_lt = []
    if xht > yht and xwd == ywd: # loop over rows
        loc_lt = oned_slide_cxscor(imgx_dat, imgy_dat, xht, yht, xwd, 'rows')
        for xloc in loc_lt:
            print(xloc)
#         divnum = xht//yht
#         stepd = mulfact*int(xht/divnum)
#         for i in range(0, divnum):
#             if (i*stepd + yht) < xht and i*(2*stepd) < xht:
#                 upperb = i*stepd + yht
#                 lowerb = i*stepd
#             else:
#                 upperb = xht
#                 lowerb = xht - yht
    elif xwd > ywd and xht == yht: # loop over columns
        loc_lt = oned_slide_cxscor(imgx_dat, imgy_dat, xwd, ywd, xht, 'cols')
        for xloc in loc_lt:
            print(xloc)
    elif xht > yht and xwd > ywd:
        mulfact = 2
        divnumh = mulfact*(xht//yht) # Make stepsize 1/2 subimage size in that vec
        divnumw = mulfact*(xwd//ywd)
        # stepht = mulfact*int(xht/divnumh)
        stepht = int(xht/divnumh)
        stepwd = int(xwd/divnumw)
        
        upperht, lowerht = 0, 0
        upperwd, lowerwd = 0, 0
        print("Ht",xht,yht," Wd", xwd, ywd)
        print("Stepht: ", stepht, "Stephwd: ", stepwd)
        for ih in range(0, divnumh):
            ''' Do stepsize everywhere, so stepht instead yht and ... '''
            temp_lt = []
            tmin_lt = []
            if (ih*stepht + yht) < xht:
                upperht = ih*stepht + yht
                lowerht = ih*stepht
            else:
                upperht = xht
                lowerht = xht - yht
            for iw in range(0, divnumw):
                if (iw*stepwd + ywd) < xwd:
#                 if (iw*stepwd + ywd) < (xwd-stepwd):
                    upperwd = iw*stepwd + ywd
                    lowerwd = iw*stepwd
                else:
                    upperwd = xwd
                    lowerwd = xwd - ywd
                temp_dat = imgx_dat[lowerht:upperht, lowerwd:upperwd]
#                 print(temp_dat.shape, type(temp_dat))
#                 temp_dat = temp_dat[:, lowerwd:upperwd]
                tempd_min = (temp_dat - temp_dat.mean())/ temp_dat.std()
                locmax = twod_slide_cxcorr(temp_dat, imgy_dat, 'rows-cols')
                locmaxnorm = twod_slide_cxcorr(tempd_min, imgy_dat, 'rows-cols')
                rowstr = str(lowerht) + ":" + str(upperht)
                colstr = str(lowerwd) + ":" + str(upperwd)
                print("Indices", rowstr, colstr, " LocMax: ", locmax, locmaxnorm)
                temp_lt.append(int(locmax))
                tmin_lt.append(int(locmaxnorm))
            locmax_lt.append(temp_lt)
            locnmax_lt.append(tmin_lt)
    for xlt in locmax_lt:
        print("Index", np.argmax(xlt), np.amax(xlt))
        print(xlt)
    for ilt in locnmax_lt:
        print("Index", np.argmax(ilt), np.amax(ilt))
        print(ilt)
#     pprint(locmax_lt)
    
#>******************************************************************************
def twod_slide_cxcorr(islice, subimg, ldirec):
#     print("slice:", islice.shape, "sub:", subimg.shape)
    img_product = fftpack.fft2(islice) * fftpack.fft2(subimg).conj()
    inv_prod = fftpack.ifft2(img_product)
    return np.amax(inv_prod.real)
#     doplt(islice)
#     return np.argmax(inv_prod.real), np.amax(inv_prod.real)
    

#>******************************************************************************
def oned_slide_cxscor(mimg_dat, simg_dat, majx, minx, equald, ldirec):
    """ In the case when the subimage has either width or height = to that of
    big image
    Perform sliding 2d cross-correlate calculation over the smaller dimension
    mimg_dat - Main image numpy data array - normalized
    simg_dat - Subimage numpy data array - normalized
    majx - size major dimension of  full image - int
    minx - size minor dimension of subimage thats < majd
    equald - size of dimension thats = between full image and subimage      """
    mulfact = 4
    upperb, lowerb = 0, 0
    divnum = mulfact*(majx//minx)
    stepd = int(majx/divnum)

    cxcorr_maxlt = []
    for i in range(0, divnum):
        if i*stepd + minx < majx:
            upperb = i*stepd + minx
            lowerb = i*stepd
        else:
            upperb = majx
            lowerb = majx - minx
        if ldirec == 'rows':
            islc = mimg_dat[lowerb:upperb]
            print("Height:", islc.shape[0], "Width:", islc.shape[1])
            img_product = fftpack.fft2(islc) * fftpack.fft2(simg_dat).conj()
            inv_prod = fftpack.ifft2(img_product)
            cxcorr_maxlt.append((i, np.amax(inv_prod.real)))
#             doplt(islc)
#             doplt(inv_prod)
        elif ldirec == 'cols':
            islc = mimg_dat[:][lowerb:upperb]
            print("Height:", islc.shape[0], "Width:", islc.shape[1])
            img_product = fftpack.fft2(islc) * fftpack.fft2(simg_dat).conj()
            inv_prod = fftpack.fft2(img_product)
            cxcorr_maxlt.append((i, np.amax(inv_prod)))
            print(np.argmax(inv_prod))
            doplt(islc)
            doplt(inv_prod)
    return cxcorr_maxlt

#     img_product = fftpack.fft2(imgx_dat) * fftpack.fft2(imgy_dat).conj()
#     cc_image = fftpack.ifft2(img_product)
#     cc_image.shape()
#     image_product = np.fft.fft2(image) * np.fft.fft2(offset_image).conj()


    
timgpath = "/Users/vkorotki/Movies/Utils/img-check-subset/Testing/"
timg1 = timgpath + 'jesusc8.jpg'
timg2 = timgpath + 'jc8slice8.jpg'
timg3 = timgpath + 'jc8slice8cut.jpg'

arg_lt = [timg1, timg2, timg3]
# arg_lt = [timg1, timg3]
# narg = [do_imgfft2(ix) for ix in arg_lt]
narg = [do_imgzncc(ix) for ix in arg_lt]
barg = [do_znccfft2(ix) for ix in arg_lt]
acombs = list(itr.combinations(list(range(0,len(arg_lt))),2))

# fft2_crosscor(narg[0], do_imgzncc(arg_lt[1]) )
for ac in acombs:
    if ac[0] == 0:
        fft2_crosscorr(narg[ac[0]],narg[ac[1]])

    
# for xdat in narg:
#     xc2d = cor2d(xdat, xdat, mode='same')
#     print(xc2d.max())
#     xdot = np.dot(xdat, xdat.T)
#     print(np.average(np.abs(xdot)))
#     doplt(xdat)
print("\n")
# for bdat in barg:
#     xc2d = cor2d(bdat, bdat, mode='same')
#     print(xc2d.max())
#     doplt(bdat)
    
 # normalize per http://en.wikipedia.org/wiki/Cross-correlation




Height: 91 Width: 600
Height: 91 Width: 600
Height: 91 Width: 600
Height: 91 Width: 600
Height: 91 Width: 600
Height: 91 Width: 600
Height: 91 Width: 600
Height: 91 Width: 600
Height: 91 Width: 600
Height: 91 Width: 600
Height: 91 Width: 600
Height: 91 Width: 600
Height: 91 Width: 600
Height: 91 Width: 600
Height: 91 Width: 600
Height: 91 Width: 600
(0, 44381.832)
(1, 61858.203)
(2, 69151.812)
(3, 60948.691)
(4, 47688.109)
(5, 29427.672)
(6, 13675.129)
(7, 8740.9746)
(8, 13780.902)
(9, 18461.961)
(10, 17378.135)
(11, 16091.141)
(12, 13339.034)
(13, 14356.913)
(14, 14356.913)
(15, 14356.913)
Ht 400 90  Wd 600 108
Stepht:  50 Stephwd:  60
Indices 0:90 0:108  LocMax:  2464.98 5949.36
Indices 0:90 60:168  LocMax:  2344.59 6889.5
Indices 0:90 120:228  LocMax:  2453.19 7104.32
Indices 0:90 180:288  LocMax:  8314.94 6351.02
Indices 0:90 240:348  LocMax:  4352.14 3078.59
Indices 0:90 300:408  LocMax:  11680.1 7391.54
Indices 0:90 360:468  LocMax:  2258.22 7076.37
Indices 0:90 420:528  LocMax: 